In [1]:
import os
import json
import numpy as np
from ultralytics import YOLO, settings
from utility_models import get_backbone_path
from config import DATASETS_YAML_LIST, ALL_MODELS, TUNING_NAMES_LIST_DEEPFISH, MODEL_BACKBONE_LAYERS
from ray import tune

/home/memorista01/miniconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-10 23:28:14,326	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-10 23:28:14,620	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
SEARCH_SPACES = {
        False: {
                'lr0': (1e-5, 1e-1),
                'lrf': (0.01, 0.5),
                'momentum': (0.6, 0.98),
                'weight_decay': (0.0, 0.001),
                'warmup_epochs': (0.0, 5.0),
                'warmup_momentum': (0.0, 0.95)
        },
        True: {
                'lr0': tune.uniform(1e-5, 1e-1),
                'lrf': tune.uniform(0.01, 0.5),
                'momentum': tune.uniform(0.6, 0.98),
                'weight_decay': tune.uniform(0.0, 0.001),
                'warmup_epochs': tune.uniform(0.0, 5.0),
                'warmup_momentum': tune.uniform(0.0, 0.95)
        }
}

In [34]:
def create_tune_dict(models, datasets, optimizers, use_ray=False, use_freeze=False):
    datasets_yaml_dir = os.path.abspath("datasets_yaml")
    
    tune_dict = {}

    for model_name in models:
        for dataset_yaml in datasets:
            # Path completo del archivo Yaml
            data_yaml = os.path.join(datasets_yaml_dir, dataset_yaml)

            for optimizer in optimizers:
                # Definir nombre del experimento
                caso = os.path.splitext(dataset_yaml)[0] + "_" + model_name + "_" + optimizer

                # Parámetros
                model = model_name
                tuning_params = {"data" :data_yaml, "optimizer": optimizer}
                
                # Guardarlos en un diccionario por caso
                tune_dict[caso] = {"model": model, "tuning_params": tuning_params, "done": False,
                                   "use_ray": use_ray, "use_freeze": use_freeze}

    return tune_dict

def do_tuning(values, epochs, iterations):
    # Cargar parámetros del diccionario
    model_name = values["model"]
    use_ray = values["use_ray"]
    data = values["tuning_params"]["data"]
    optimizer = values["tuning_params"]["optimizer"]
    
    # Actualizar los parámetros de entrenamiento
    train_params = {"space": SEARCH_SPACES[use_ray], "epochs": epochs, "iterations": iterations,
                    "use_ray": use_ray, "data": data, "optimizer": optimizer}
    if values["use_freeze"]:
        train_params.update(freeze=MODEL_BACKBONE_LAYERS[model_name])
    if use_ray:
        train_params.update(gpu_per_trial=1)

    # Cargar modelo
    model = YOLO(get_backbone_path(model_name), task="segment")
    
    # Realizar tuning
    result_grid = model.tune(**train_params)
        

def run_tuning_file(json_file, epochs, iterations):
    # Cargar el archivo de configuración para el tuning
    tune_dict = load_tune_config(json_file)

    for case, values in tune_dict.items():
        if values["done"]:
            continue

        do_tuning(values, epochs, iterations)
        
        tune_dict[case]["done"] = True
        save_tune_config(tune_dict, json_file)


def save_tune_config(tune_dict, json_file):
    # Crea el directorio si no existe
    directorio = os.path.dirname(json_file)
    if directorio and not os.path.exists(directorio):
        os.makedirs(directorio)

    with open(json_file, 'w', encoding='utf-8') as archivo:
        json.dump(tune_dict, archivo, ensure_ascii=False, indent=4)


def load_tune_config(json_file) -> dict:
    with open(json_file, 'r', encoding='utf-8') as archivo:
        tune_dict = json.load(archivo)
    return tune_dict

In [35]:
models = ['yolov8n-seg']
datasets = ['Deepfish.yaml']
optimizers = ['SGD']

tune_dict_1 = create_tune_dict(models, datasets, optimizers, False, False)
tune_dict_2 = create_tune_dict(models, datasets, optimizers, True, False)

In [36]:
tuning_config_deepfish_1 = "tuning/testing_json_1"
tuning_config_deepfish_2 = "tuning/testing_json_2"

In [37]:
save_tune_config(tune_dict_1, tuning_config_deepfish_1)
save_tune_config(tune_dict_2, tuning_config_deepfish_2)

In [38]:
run_tuning_file(tuning_config_deepfish_1, 20, 5)

Tuner: Initialized Tuner instance with 'tune_dir=runs/segment/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/5 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8}


New https://pypi.org/project/ultralytics/8.3.107 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 11912MiB)
engine/trainer: task=segment, mode=train, model=models/backbone/yolov8n-seg.pt, data=/home/memorista01/tesis/datasets_yaml/Deepfish.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=N

train: Scanning /home/memorista01/tesis/datasets_yolo/Deepfish/labels/train.cache... 148 images, 161 backgrounds, 0 corrupt: 100%|██████████| 309/309 [00:00<?, ?it/s]
val: Scanning /home/memorista01/tesis/datasets_yolo/Deepfish/labels/valid.cache... 66 images, 59 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<?, ?it/s]


Plotting labels to runs/segment/train/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/20      2.52G      1.316      2.446      4.447      1.295          8        640: 100%|██████████| 20/20 [00:03<00:00,  5.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.57it/s]
  0%|          | 0/20 [00:00<?, ?it/s]

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 2/4 [00:00<00:00,  4.18it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.32it/s]


                   all        125         79      0.986      0.898      0.936      0.739      0.997      0.924       0.96      0.724
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/segment/train5
💡 Learn more at https://docs.ultralytics.com/modes/train
Saved runs/segment/tune/tune_scatter_plots.png
Saved runs/segment/tune/tune_fitness.png

Tuner: 5/5 iterations complete ✅ (437.92s)
Tuner: Results saved to runs/segment/tune
Tuner: Best fitness=1.50726 observed at iteration 5
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.98548, 'metrics/recall(B)': 0.89873, 'metrics/mAP50(B)': 0.93567, 'metrics/mAP50-95(B)': 0.73794, 'metrics/precision(M)': 0.99655, 'metrics/recall(M)': 0.92405, 'metrics/mAP50(M)': 0.95966, 'metrics/mAP50-95(M)': 0.7262, 'val/box_loss': 0.80435, 'val/seg_loss': 1.36283, 'val/cls_loss': 0.96369, 'val/dfl_loss': 0.93132, 'fitness': 1.50726}
Tuner: Best fitness model is runs/segment/train5
Tuner: Best fitn

In [33]:
run_tuning_file(tuning_config_deepfish_2, 20, 5)

(_tune pid=136939) New https://pypi.org/project/ultralytics/8.3.107 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=136939) Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 11912MiB)
(_tune pid=136939) engine/trainer: task=segment, mode=train, model=models/backbone/yolov8n-seg.pt, data=/home/memorista01/tesis/datasets_yaml/Deepfish.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visu

  0%|          | 0.00/5.35M [00:00<?, ?B/s]
 19%|█▊        | 1.00M/5.35M [00:00<00:00, 10.1MB/s]
 44%|████▍     | 2.38M/5.35M [00:00<00:00, 11.9MB/s]
 68%|██████▊   | 3.62M/5.35M [00:00<00:00, 11.8MB/s]
100%|██████████| 5.35M/5.35M [00:00<00:00, 11.7MB/s]


(_tune pid=136939) AMP: checks passed ✅


train: Scanning /home/memorista01/tesis/datasets_yolo/Deepfish/labels/train.cache... 148 images, 161 backgrounds, 0 corrupt: 100%|██████████| 309/309 [00:00<?, ?it/s]
val: Scanning /home/memorista01/tesis/datasets_yolo/Deepfish/labels/valid.cache... 66 images, 59 backgrounds, 0 corrupt: 100%|██████████| 125/125 [00:00<?, ?it/s]


(_tune pid=136939) Plotting labels to runs/segment/train/labels.jpg... 
(_tune pid=136939) optimizer: SGD(lr=0.09220505845150082, momentum=0.8822568681044903) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0007766144966996587), 76 bias(decay=0.0)
(_tune pid=136939) Image sizes 640 train, 640 val
(_tune pid=136939) Using 8 dataloader workers
(_tune pid=136939) Logging results to runs/segment/train
(_tune pid=136939) Starting training for 20 epochs...
(_tune pid=136939) 
(_tune pid=136939)       Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

  0%|          | 0/20 [00:00<?, ?it/s]
      19/20      2.78G     0.8931      1.264      1.208     0.9335         10        640:   5%|▌         | 1/20 [00:00<00:02,  7.07it/s]
      19/20      2.78G      1.024      1.094      1.418      1.014          7        640:  10%|█         | 2/20 [00:00<00:02,  7.07it/s]
      19/20      2.78G      1.059      1.265      1.339      1.026          9        640:  15%|█▌        | 3/20 [00:00<00:02,  7.06it/s]
      19/20      2.78G      1.092      1.332       1.48      1.008          7        640:  20%|██        | 4/20 [00:00<00:02,  6.80it/s]
      19/20      2.78G      1.084      1.318      1.467      1.021         15        640:  25%|██▌       | 5/20 [00:00<00:02,  6.89it/s]
      19/20      2.78G      1.074      1.351      1.415      1.024         13        640:  30%|███       | 6/20 [00:00<00:02,  6.91it/s]
      19/20      2.78G      1.088      1.408      1.402      1.055          9        640:  35%|███▌      | 7/20 [00:01<00:01,  6.97it/s]
  

(_tune pid=138894)                    all        125         79      0.983      0.582      0.779        0.5      0.837      0.658      0.807      0.531
(_tune pid=138894) 
(_tune pid=138894)       Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/20 [00:00<?, ?it/s]
      20/20      2.78G     0.9978      1.202       1.32       1.02          8        640:   5%|▌         | 1/20 [00:00<00:02,  7.14it/s]
      20/20      2.78G      1.059      1.114      1.443      1.069          6        640:  10%|█         | 2/20 [00:00<00:02,  7.10it/s]
      20/20      2.78G      1.132       1.19      1.441      1.088          9        640:  15%|█▌        | 3/20 [00:00<00:02,  7.07it/s]
      20/20      2.78G      1.061      1.116      1.368      1.045         12        640:  20%|██        | 4/20 [00:00<00:02,  6.79it/s]
      20/20      2.78G     0.9929      1.146      1.349      1.011          9        640:  25%|██▌       | 5/20 [00:00<00:02,  6.88it/s]
      20/20      2.78G      1.039       1.29      1.404      1.052         11        640:  30%|███       | 6/20 [00:00<00:02,  6.93it/s]
      20/20      2.78G      1.024      1.296      1.347      1.051         15        640:  35%|███▌      | 7/20 [00:01<00:01,  6.94it/s]
  

(_tune pid=138894)                    all        125         79      0.986      0.867      0.936      0.658      0.986      0.867      0.939      0.658


2025-04-10 23:48:51,573	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/memorista01/tesis/runs/detect/tune/_tune_2025-04-10_23-42-11' in 0.0381s.
2025-04-10 23:48:51,577	INFO tune.py:1041 -- Total run time: 400.19 seconds (400.12 seconds for the tuning loop).
